# Download the models and make disease risk prediction

Prepare OTU relative abundance table (rows as species represented by ncbi taxonmy id ) and download the disease-prediction model（e.g. D000067877） to predict the risk.


Uploading species abundance table to the [web-server](http://39.100.246.211:8051/balance/server/) to make risk prediction is suggested.

Ranking risk probabilities with visualization is supported on the server.


In [ ]:
from joblib import load
import pandas as pd
import numpy as np


healthID = "D006262"
diseaseID = "D000067877"  
##### otu2sbp balance matrix

def fromContrast(x, contrasti):  
    '''calculate balance i, contrasti is the ith column in the sbp matrix'''    
    lpos = np.sum(contrasti == 1)
    lneg = np.sum(contrasti == -1)
    const = np.sqrt((lpos*lneg)/(lpos+lneg))    
    logX = np.log(x)   
    ipos = np.mean(logX.loc[:,contrasti == 1],axis=1)
    ineg = np.mean(logX.loc[:,contrasti == -1],axis=1)
    balancei = const * np.log(np.exp(ipos) / np.exp(ineg)) #balancei 在所有样品中的值,一列
    return balancei    
    

def otu2balance(otu, sbp):
    # test otu table需要根据sbp矩阵转化成与sbp中otu数量和顺序一致的的 new otutable x
    x = pd.DataFrame(index=otu.index, columns=sbp.index)
    x[[i for i in sbp.index if i in otu.columns]] = otu[[i for i in sbp.index if i in otu.columns]] #保留overlap的特征
    x[[i for i in sbp.index if i not in otu.columns]] = 0.0
    x[x==0] = 0.0001     
    balance = sbp.apply(lambda col : fromContrast(x, col), axis=0) #calculate balances
    return balance
 
    
#sbp matrix 和 model 一一对应，    
model="./Data/metagenome/D000067877/otu_table.xls.balance.csv.NonScaler_LogisticRegression.model.z"
sbpf = model.replace('csv.NonScaler_LogisticRegression.model.z', 'sbp.csv') 

m=load(model)
sbp=pd.read_csv(sbpf, index_col=0, header=0)

## OTU table, rows as ncbi taxids (OTUs), columns as samples 
otu_tablef = './Data/metagenome/demo_otu_table20.xls'
otu_table = pd.read_csv(otu_tablef,sep="\t",index_col=0, header=0)
otu_table = otu_table.T 


#根据sbp matrix将otu table转化成模型输入要求的 balance table
balance_table = otu2balance(otu_table,sbp)

# predict risks
proba=m.predict_proba(balance_table)

# probability to dataframe
diseaseCol = 1 if diseaseID > healthID else 0 # ascII值比大小
disease_proba = [i[diseaseCol] for i in proba]
disease_risk = pd.DataFrame({'SampleID': balance_table.index,
                             '{} Risk Probability'.format(diseaseID): disease_proba})

# disease_resk 
'''
>>> disease_risk
    SampleID            D000067877 Risk Probability
0   ERR2238626          0.000958
1   ERR2238632          0.019925
2   ERR2238634          0.294495
3   ERR2238647          0.052416
4   ERR2238648          0.151595
5   ERR2238665          0.253899
6   ERR2238676          0.071860
7   ERR2238682          0.461082
8   ERR2238687          0.143696
9   ERR2238694          0.559140
10  ERR2238695          0.010018
11  ERR2238705          0.143426
12  ERR2238712          0.013632
13  ERR2238715          0.089387
14  ERR2238721          0.143459
15  ERR2238738          0.814422
16  ERR2238745          0.008284
17  ERR2238748          0.262607
18  ERR2238754          0.013382
'''